In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

## Get article titles and teaser text 

In [ ]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [ ]:
html = browser.html  #get all the html
soup = BeautifulSoup(html, 'html.parser')

#testing output, see what to get
#print(soup.prettify())
 



#The below will get you *all* the titles and articles. The final requirements only have 1
# titles=[]
# teaser_texts=[]
# divs = soup.find_all('div')
# for div in divs:
#     #print(div['class'][0])  #this will get you content_title and article_teaser_body
#     if div['class'][0]=='content_title':
#         #print(div.text)
#         titles.append(div.text)
#     if div['class'][0]=='article_teaser_body':
#         #print(div.text)
#         teaser_texts.append(div.text)
#print(titles[0])
#print(teaser_texts[0])

title = soup.find('div', class_='content_title').text
teaser_text = soup.find('div', class_='article_teaser_body').text
print(teaser_text)


In [ ]:
# NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch
# There are lots of ways to participate in the historic event, which is targeted for July 30.

## Get featured image url

In [ ]:
iurl = 'https://spaceimages-mars.com/'
browser.visit(iurl)

In [ ]:
ihtml = browser.html
isoup = BeautifulSoup(ihtml, 'html.parser')

#checkout html output, looking for featured image 
#print(isoup.prettify())

# looking for this src:
# <img class="headerimage fade-in" src="image/featured/mars1.jpg">

imgs = isoup.find_all('img')
for img in imgs:
    #print(img) #testing message
    try:
        if img['class'][0]=='headerimage':
            #print(img['src'])  #testing message
            featured_image_file = img['src']
            break   #header image will be towards the top, once found don't need to continue the loop
    except:  #if there's no class, continue to the next one
        continue

#concat the url to the image location
featured_image_url = iurl+featured_image_file

In [ ]:
#check the url value
featured_image_url

## Get Mars facts

In [ ]:
turl = 'https://galaxyfacts-mars.com/'

In [ ]:
tables = pd.read_html(turl)
tables[1]

In [ ]:
mars_earth_df = tables[0]
mars_facts_df = tables[1]

In [ ]:
mars_facts_df.rename(columns={0:"FACT_DESC",1:"FACT_VALUE"}, inplace=True)

In [ ]:
mars_earth_df.rename(columns={0:"ATTRIBUTE",1:"MARS",2:"EARTH"}, inplace=True)
mars_earth_df

In [ ]:
#delete the 1st row which is more of a table header
mars_earth_df =mars_earth_df.drop(0)  
mars_earth_df  #not bothering to reindex because not keeping it in the html

In [ ]:
#push dfs to html files
mars_earth_df.to_html('mars_earth_comp.html', index=False)
mars_facts_df.to_html('mars_facts.html', index=False)

## Get hemisphere info

In [ ]:
hurl = 'https://marshemispheres.com/'
browser.visit(hurl)
hemisphere_image_urls = []

In [ ]:
html = browser.html  #get all the html
hsoup = BeautifulSoup(html, 'html.parser')

In [ ]:
#need to get the list of 4 headers from main page and then loop through each.
list_of_subpages = []

a_itemlinks = hsoup.find_all('a', class_='itemLink')

for alink in a_itemlinks:
    #print(alink['href']) #testing output
    cur_href = alink['href']
    if 'html' in cur_href:
        #print('it does') #testing output
        if cur_href not in list_of_subpages:
            #print('add it!') #testing output
            list_of_subpages.append(cur_href)

list_of_subpages

In [ ]:
for page in list_of_subpages:
    sub_url=hurl+page
    #print(sub_url) #testing output
    
    browser.visit(sub_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify()) #testing output
    
    #looking for this <img class="wide-image" src="images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"/>
    img = soup.find('img', class_='wide-image')
    title=soup.find('h2', class_='title').text.split(" Enhanced")[0]
    #print(hurl+img['src'])  #testing output
    hemis_dict = {}
    hemis_dict['title'] = title
    hemis_dict['img_url'] = hurl+img['src']
    #print(hemis_dict)  #testing output
    
    hemisphere_image_urls.append(hemis_dict)

    #another way to do this would've been to start in landing page, use a click to get to each subpage and then would use the below click to go back
    #browser.links.find_by_partial_text('Back').click()
    #break #testing break. got code working for 1st page then removed this

In [ ]:
#check the dictionary is properly populated
hemisphere_image_urls


#testing output, see what to get
#print(hsoup.prettify())

#print(a_itemlinks.find('img'))
#img_info=a_itemlinks.find('img')

#this is getting the thumbnail off the main page
# hemis_dict = {}
# hemis_dict['title'] = img_info['alt'].split(" Enhanced")[0]
# hemis_dict['img_url'] = hurl+img_info['src']
# img_info['alt'].split(" Enhanced")[0]  #title!  yes, there are other ways to get this. could take if off 1st page too
# hurl+img_info['src']
#browser.links.find_by_partial_text('Back').click()

In [ ]:
for x in range(1):
    #browser.links.find_by_partial_text('Hemisphere').click()  #partial text needs to be a variable from loop
    browser.links.find_by_partial_text('Schiaparelli').click() 
    lhtml = browser.html  #get all the html
    lsoup = BeautifulSoup(lhtml, 'html.parser')
    #testing output, see what to get
    print(lsoup.prettify())
    print('--------------------------------------------------------------')
    a_itemlinks = lsoup.find('a', class_='itemLink')
    #h3s = hsoup.find_all('h3')
    print(a_itemlinks.find('img'))
    img_info=a_itemlinks.find('img')
    print('fucking'+img_info['alt'])
    hemis_dict = {}
    hemis_dict['title'] = img_info['alt'].split(" Enhanced")[0]
    hemis_dict['img_url'] = hurl+img_info['src']
    #img_info['alt'].split(" Enhanced")[0]  #title!  yes, there are other ways to get this. could take if off 1st page too
    #hurl+img_info['src']
    hemisphere_image_urls.append(hemis_dict)
    
    
    #browser.links.find_by_partial_text('Back').click() 

In [ ]:
hemis_dict
hemisphere_image_urls

In [ ]:
browser.quit()